# pdfダウンロード テキスト変換 txt出力

In [ ]:
!pip install tika

In [ ]:
import pandas as pd
import datetime
import subprocess
import os
from tika import parser
import glob
from tqdm import tqdm
import urllib

In [ ]:
pdf_df = pd.read_csv('/content/drive/My Drive/covid_guideline_study/covid_guideline_list_1030.csv')
pdf_df

,title,url,cat
0,全国公民館連合会,https://www.kominkan.or.jp/file/all/2020/2020...,art
1,日本クラシック音楽事業協会,https://storage.googleapis.com/classicorjp-pu...,art
2,緊急事態舞台芸術ネットワーク,http://jpasn.net/stage_guideline0918d.pdf,art
3,全日本合唱連盟,https://jcanet.or.jp/JCAchorusguideline-ver1_...,art
4,全興連映画館,https://www.zenkoren.or.jp/news-pdf/0919-1_CO...,art
5,全興連演芸場,https://www.zenkoren.or.jp/news-pdf/0919-2_CO...,art
6,コンサートプロモーターズ協会,http://www.acpc.or.jp/pdf/COVID-19/20201008_0...,art
7,日本音楽事業者協会,https://www.jame.or.jp/wordpress/wp-content/u...,art
8,日本音楽制作者連盟,http://www.fmp.or.jp/information/wp-content/u...,art
9,全国公民館連合,https://www.kominkan.or.jp/file/all/2020/2020...,ass


In [ ]:
# urllibによるdownloadの関数を定義 https://note.nkmk.me/python-download-web-images/ より
# errorの際、エラーコードと失敗したファイル名を表示するように改変
def download_file(url, dst_path, filename):
    try:
        with urllib.request.urlopen(url) as web_file:
            data = web_file.read()
            with open(dst_path, mode='wb') as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print('-------', e, f'Cannot download {filename}')

In [ ]:
# 以下、csvファイル内のpdfファイルを一括ダウンロード urllibバージョン
dir_name = '/content/drive/My Drive/covid_guideline_study/covid_pdfs_1030_3'
#dir_name = 'test_urllib'
os.makedirs(dir_name, exist_ok=True)

for i in tqdm(range(len(pdf_df))):
  # ファイル名を設定：（発行団体名）_（ダウンロード日）.pdf
  filename = f'{pdf_df.iloc[i][0]}_{str(datetime.date.today())}.pdf'
  filepath = dir_name+'/'+filename
  url = pdf_df.iloc[i][1]
  download_file(url, filepath, filename)

 78%|███████▊  | 35/45 [01:29<00:16,  1.62s/it]

------- HTTP Error 403: Forbidden cannot download 全国麻雀業組合_2020-10-30.pdf


100%|██████████| 45/45 [01:48<00:00,  2.41s/it]


In [ ]:
# 以下、ダウンロードしたpdfファイルを一括してテキスト抽出しそれをtxtファイルに書き出し
pdf_paths = glob.glob(f'{dir_name}/*.pdf')
pdf_names = os.listdir(dir_name)
txt_dir_name = '/content/drive/My Drive/covid_guideline_study/covid_txts_1030'
os.makedirs(txt_dir_name, exist_ok=True)

for i in tqdm(range(len(pdf_names))):
  # pdfからテキスト情報のみを取得
  file_data = parser.from_file(pdf_paths[i])
  text = file_data["content"]
  # テキストから無用な改行とスペースを削除
  # テキスト抽出失敗の場合はファイル名を表示する
  try:
    text = text.replace('\n', '')
    text = text.replace(' ', '')
  except:
    print(f'------ Cannot extract from {pdf_names[i]}')
  # テキストファイルとして保存
  txt_path = txt_dir_name + '/' + pdf_names[i].replace('.pdf', '.txt')
  try:
    with open(txt_path, mode='w') as f:
      f.write(text)
  except:
    print(f'------ Cannot save {txt_path}')

 67%|██████▋   | 30/45 [00:07<00:02,  5.69it/s]

cannot extract from 日本アミューズメント産業協会_2020-10-30.pdf
cannot extract from 日本アミューズメント産業協会_2020-10-30.pdf


 82%|████████▏ | 37/45 [00:07<00:00,  9.06it/s]

cannot extract from 全国麻雀業組合_2020-10-30.pdf
cannot extract from 全国麻雀業組合_2020-10-30.pdf


100%|██████████| 45/45 [00:08<00:00,  5.21it/s]


In [ ]:
#!rm -r '/content/drive/My Drive/covid_guideline_study/covid_txts_1030'
#!rm -r '/content/drive/My Drive/covid_guideline_study/covid_pdfs_1030'